In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from utils import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option("display.max_columns", None)

# Daten laden und initial angucken

Als einen ersten Schritt laden wir den neuen Datensatz mit nun allen Windenergieanlagen aus dem Marktstammdatenregister. Dieser liegt vorgefärtigt in einem für Tabellen speziellen Format vor: Parquet. 

Mit dem Ausführen der unteren Zelle lädst die Daten in Pandas ein, wie du es auch schon in der letzten Übung gemacht hast. Zusätzlich führen wir auch gleich `.info()` mit aus, um Informationen zu unseren Spalten zu erhalten.

In [ ]:
MaStRWind = pd.read_parquet("MaStRWind.parquet")
MaStRWind.info()

Das sind wirklich verdammt viele Spalten... Ich kann mir unter vielen der Namen nicht einmal vorstellen was da alles hinter stecken soll!

Zum Glück gibt es da eine weitere hilfreiche Funktion in Pandas: `.head(n)`

Hiermit kannst du dir die n ersten Zeilen deiner Tabelle ausgeben lassen, also den Kopf. Logisch, wie englisch "head" auch schon sagt. Wollen wir also die ersten 10 Zeilen sehen, schreiben wir `.head(n=10)`. Wollen wir die ersten 50 Zeilen haben, dann `.head(n=50)`. Das gleich geht auch wenn du dir die letzten Zeilen von unten angucken willst. Dafür kannst du `.tail(n)` benutzen. Willst du also die 10 untersten Zeilen sehen, was bentzt du dann? Probier es aus!

In [ ]:
MaStRWind.head(n=5)

In [ ]:
MaStRWind = Datenvorbereitung(MaStRWind)

# Daten visualisieren und analysieren

Ich weiß ja nicht wie es euch geht, aber ich kann mit so riesen Tabellen immer nicht so gut arbeiten. Mit fällt es immer leichter wenn ich Bilder vor mir habe um Sachen zu erkennen. Falls es euch da ähnlich geht, haben wir in den nächsten Zellen etwas mit dem ihr ein wenig rum probieren könnt. 

### Wochenaktivitäten

Als kleiner Spaß, lasst uns dochmal gucken an welchen Wochentagen eigentlich die meisten Windenergieanlagen je Bundesland in Betrieb genommen werden. 

In [ ]:
MaStRWind_grouped = MaStRWind.loc[MaStRWind["Inbetriebnahmejahr"]>1990].groupby(by=["Bundesland","Inbetriebnahmewochentag"]).sum()["Anlagenzahl"].reset_index()
#MaStRWind_grouped["Anlagenzahl"] = MaStRWind_grouped["Anlagenzahl"] / 1000000
MaStRWind_grouped_pivot = MaStRWind_grouped.pivot(columns='Inbetriebnahmewochentag',index='Bundesland',values='Anlagenzahl')

wochentage = ["Montag","Dienstag","Mittwoch","Donnerstag","Freitag","Samstag","Sonntag"]

plt.figure(figsize=(8,8))
ax = sns.heatmap(MaStRWind_grouped_pivot, annot=True, fmt=".2f", cbar_kws={'label': 'In Betrieb genommene Windenergieanlagen je Wochentag [-]'})
ax.set_xticklabels(wochentage)
plt.xticks(rotation = 45)
plt.show()

Also am Donnerstag scheint überall am meisten los zu sein. Vielleicht wollen die Leute alle am Freitag nur noch ihre Baustelle aufräumen um dann gut ins Wochenende zu starten? Ich weiß es nicht... Und das warum das so ist, sehen wir das in den Daten? Vielleicht könnt ihr uns hierzu eure Gedanken teilen? Und was ist eigentlich mit den Windenergieanlagen Offshore los?

### Ausbau der Bundesländer seit 1990 

Als nächstes wollen wir uns einmal den Ausbau seit 1990 angucken. Hierfür gucken wir uns wieder alle Bundesländer und den Bereich Offshore einzeln an. Genauer gucken wir uns aber jetzt nicht die reine Anlagenzahl, sonder die installierte Leistung an. Also die Leistung aller in einem Jahr und Bundesland in Betrieb genommenen Windenergianlagen zusammen. Das wird sicher ein ganz schönes Brett an Bild...

In [ ]:
MaStRWind_grouped = MaStRWind.loc[MaStRWind["Inbetriebnahmejahr"]>1990].groupby(by=["Bundesland","Inbetriebnahmejahr"]).sum()["Nettonennleistung"].reset_index()
MaStRWind_grouped["Nettonennleistung"] = MaStRWind_grouped["Nettonennleistung"] / 1000
MaStRWind_grouped_pivot = MaStRWind_grouped.pivot(columns='Inbetriebnahmejahr',index='Bundesland',values='Nettonennleistung')

plt.figure(figsize=(30,8))
sns.heatmap(MaStRWind_grouped_pivot, annot=True, fmt=".2f", cbar_kws={'label': 'Zubau an Leistung [MW/a]'}, cmap="viridis")

Was war denn da ab 2020? Da ging es ja plötzlich gar nicht mehr vorran... Könnt ihr mir vielleicht helfen was dafür der Grund sein könnte?

### Räumliche Zusammenhänge

Jetzt wollen wir dochmal gucken, wie die Windenergieanlagen in Deutschland so verteil sind. Speziell wo große Anlagen stehen und wo kleine. Dafür plotten wir alle uns zur Verfügung stehenden Analgen einmal über ihre Koordinaten, lassen die größe der Punkte von den Rotordurchmessern bestimmt sein und die Farbe nach den Bundesländer. Ich bin mal gespannt wie das so aussieht!

In [ ]:
plt.figure(figsize=(12,12))
sns.scatterplot(
    data=MaStRWind.loc[MaStRWind["Inbetriebnahmejahr"]>1990], 
    x="Laengengrad", y="Breitengrad", 
    hue="Bundesland", size="Rotordurchmesser", #style="Bundesland", 
    palette="husl"
)

Huch! Da sind aber einige Anlagen dabei, die garnicht in Deutschland stehen... 

In [ ]:
plt.figure(figsize=(12,12))
sns.scatterplot(
    data=MaStRWind.loc[MaStRWind["Inbetriebnahmejahr"]>1990], 
    x="Laengengrad", y="Breitengrad", 
    hue="Nabenhoehe", size="Rotordurchmesser", style="Bundesland", 
    palette="husl", sizes=(10, 150)
)

In [ ]:
plt.figure(figsize=(16,12))
sns.scatterplot(
    data=MaStRWind.loc[(MaStRWind["Inbetriebnahmejahr"]>1990) & (MaStRWind["Bundesland"]!="Seelage")], 
    x="Inbetriebnahmedatum", y="Nabenhoehe", 
    hue="Bundesland", size="Rotordurchmesser", sizes=(20, 200)
)

In [ ]:
MaStRWind_wms = MaStRWind[MaStRWind["Rotordurchmesser"].notna()]
MaStRWind_wms = MaStRWind_wms.loc[MaStRWind_wms["Inbetriebnahmejahr"]>1990]
MaStRWind_wms.sort_values(by=["Inbetriebnahmedatum"], inplace=True)

fig = go.Figure(px.scatter_mapbox(MaStRWind_wms, 
            lat="Breitengrad", 
            lon="Laengengrad", 
            color="Nettonennleistung",
            #size="Rotordurchmesser",
            #size_max=40,
            hover_name="EinheitMastrNummer", 
            hover_data=["Inbetriebnahmedatum","Hersteller","Typenbezeichnung","Nettonennleistung","Nabenhoehe","Rotordurchmesser"],
            color_continuous_scale="jet", 
            zoom=4,
            height=900,
            mapbox_style="open-street-map",
            #animation_frame="Inbetriebnahmejahr",
            )
        )      

fig.update_layout(
    margin = {'l':0, 'r':0, 'b':0, 't':0},
    coloraxis_colorbar=dict(
        title="Nettonennleistung [kW]",
    ),
    title="Windenergieanlagenbestand nach Leistung und Rotordurchmesser",
)

fig.show()